In [1]:
import pandas as pd
import numpy as np
import string
import random as rn

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20,20)

from IPython.display import SVG
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import scipy.sparse as sparse
import pickle
import csv
import implicit
import itertools
import copy
plt.style.use('ggplot')


Using TensorFlow backend.


In [2]:
#get recipe data
recipe_all = recipe_all = pd.read_csv('/Users/xun/Documents/Thesis/recipe_data/recipe_data_processed.csv') 
recipes = recipe_all.iloc[:,0:3]
recipes['publishdate'] = recipes['publishdate'].astype('datetime64')
recipes.head()

,publishdate,content_id,title_en
0,2017-02-02,R574,Moroccan-inspired chickpea stew
1,2017-11-05,R943,Lamb and quince tagine
2,2017-10-16,C27,Fudgy cardamom banana bread
3,2017-04-29,R702,Braised chicken with couscous
4,2018-07-24,R1295,East African-style coconut chicken


In [3]:
df_all = pd.read_csv('/Users/xun/Documents/Thesis/interaction_data/ks-18m-cooks-v1.csv')
df_all = df_all.sort_values(by = ['USERID', 'TIMESTAMP'])
df_all = df_all.reset_index(drop=True)

df = df_all.iloc[:, 0:3]
df.columns = ['USERID', 'content_id', 'TIMESTAMP']
df.TIMESTAMP = df.TIMESTAMP.astype('datetime64')
df.head()

,USERID,content_id,TIMESTAMP
0,1735967714,R755,2017-11-15 19:20:53.731
1,1735967714,R1017,2017-11-28 17:41:43.973
2,1735967714,R625,2017-11-29 17:54:51.629
3,1735967714,R302,2017-11-29 17:55:28.847
4,1735967714,R638,2018-01-23 07:39:59.955


In [4]:
df = df.merge(recipes, 'left', on = 'content_id' )
df_o = df
df = df[['USERID', 'title_en', 'TIMESTAMP']]
df.head()

,USERID,title_en,TIMESTAMP
0,1735967714,Fettuccine Alfredo,2017-11-15 19:20:53.731
1,1735967714,Zurich-style veal and mushroom ragout,2017-11-28 17:41:43.973
2,1735967714,Shakshuka bowl,2017-11-29 17:54:51.629
3,1735967714,Shakshuka,2017-11-29 17:55:28.847
4,1735967714,Thai peanut-coconut soup with rice noodles,2018-01-23 07:39:59.955


In [7]:
df = df[~df.title_en.isna()]

In [8]:
# 39648 users, 1743 items, 859766 cooks
len(list(df['USERID'].unique()))
len(list(df['title_en'].unique()))

39648

1743

In [9]:
# Create mappings
index_to_recipe = np.load('/Users/xun/Documents/Thesis/recipe_data/index_to_recipe.npy', allow_pickle=True).item()
recipe_to_index = np.load('/Users/xun/Documents/Thesis/recipe_data/recipe_to_index.npy', allow_pickle=True).item()
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df.USERID.unique().tolist()):
    uid_to_idx[uid] = idx
    idx_to_uid[idx] = uid

In [10]:
len(recipe_to_index)
len(uid_to_idx)

1814

39648

In [11]:
def map_ids(row, mapper):
    return mapper[row]

In [12]:

I = df.USERID.apply(map_ids, args=[uid_to_idx]).as_matrix() # turn uid to idx
J = df.title_en.apply(map_ids, args=[recipe_to_index]).as_matrix() # turn mid to idx
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
df['uid'] = df.USERID.apply(map_ids, args=[uid_to_idx]).values
df['rid'] = df.title_en.apply(map_ids, args=[recipe_to_index]).values
df.head()

,USERID,title_en,TIMESTAMP,uid,rid
0,1735967714,Fettuccine Alfredo,2017-11-15 19:20:53.731,0,808
1,1735967714,Zurich-style veal and mushroom ragout,2017-11-28 17:41:43.973,0,570
2,1735967714,Shakshuka bowl,2017-11-29 17:54:51.629,0,1676
3,1735967714,Shakshuka,2017-11-29 17:55:28.847,0,901
4,1735967714,Thai peanut-coconut soup with rice noodles,2018-01-23 07:39:59.955,0,1795


In [14]:
recipes['rid'] = recipes.title_en.apply(map_ids, args=[recipe_to_index]).values
recipes.head()

,publishdate,content_id,title_en,rid
0,2017-02-02,R574,Moroccan-inspired chickpea stew,0
1,2017-11-05,R943,Lamb and quince tagine,1
2,2017-10-16,C27,Fudgy cardamom banana bread,2
3,2017-04-29,R702,Braised chicken with couscous,3
4,2018-07-24,R1295,East African-style coconut chicken,4


In [15]:
I.shape

(859766,)

In [16]:
def train_test_split(ratings, df, split_count, fraction=None):
    """
    Split recommendation data into train and test sets
    
    Params
    ------
    ratings : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all 
        users are considered.
    """
    # Note: likely not the fastest way to do things below.
    train = ratings.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
#     numpy.random.seed(0)
    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*k, fraction))
            raise
    else:
        user_index = range(train.shape[0])
        
    train = train.tolil()

    for user in user_index:
        test_ratings = df.loc[df['uid']==user][-split_count:]['rid'].values
#         test_ratings = np.random.choice(ratings.getrow(user).indices, 
#                                         size=split_count, 
#                                         replace=False)
        train[user, test_ratings] = 0.
        # These are just 1.0 right now
        test[user, test_ratings] = ratings[user, test_ratings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), user_index

In [17]:
np.random.seed(1234)
train, test, user_index = train_test_split(likes, df, 10, fraction=0.1)

In [18]:
user_index[0:5]

[34899, 20983, 22232, 15600, 2931]

In [20]:
train.todense().shape, test.todense().shape, len(user_index)

((39648, 1815), (39648, 1815), 3964)

In [21]:
user = train.nonzero()[0]
item = train.nonzero()[1]
train_df = pd.DataFrame(data=np.column_stack((user,item)),columns=['uid','idx'])
# train_df = train_df.merge(mapping, 'inner', on= 'idx')
train_df.shape

(820126, 2)

In [22]:
user_test = test.nonzero()[0]
item_test = test.nonzero()[1]
test_df = pd.DataFrame(data=np.column_stack((user_test,item_test)),columns=['uid','idx'])
# test_df = test_df.merge(mapping, 'inner', on= 'idx')
test_df.shape

(39640, 2)

In [23]:
import implicit 

model_als = implicit.als.AlternatingLeastSquares(factors = 50, regularization = 0.01, 
                                                         iterations = 100)

In [24]:
# train is user_items sparse matrix, model.fit(item_users)
# training time cooks_18m : 02:09
model_als.fit(train.T)

100%|██████████| 100.0/100 [02:08<00:00,  1.10s/it]


In [75]:
# Recommends items for a user
recommendations = model_als.recommend(24687, train,  N=10, filter_already_liked_items=False)



In [76]:
recommendations#[0][0]

[(1698, 1.0726285),
 (610, 1.0016286),
 (674, 0.95832396),
 (330, 0.951195),
 (901, 0.951151),
 (783, 0.90967643),
 (883, 0.84464735),
 (933, 0.8245903),
 (1082, 0.7832339),
 (794, 0.5095008)]

In [77]:
def get_rec_id(list):
    ids = []
    for item in list:
        item = item[0]
        ids.append(item)
    return ids
get_rec_id(recommendations) 

[1698, 610, 674, 330, 901, 783, 883, 933, 1082, 794]

In [78]:
def get_rec_title(list):
    titles = []
    for item in get_rec_id(list):
        title = index_to_recipe[item]
        titles.append(title)
    return titles
get_rec_title(recommendations)

['Chili con carne',
 'Sweet potato curry',
 'Four-cheese and tomato baked penne',
 'Oven ratatouille with feta',
 'Shakshuka',
 'One-pot spinach pasta',
 'Hot chickpea soup',
 'Creamy whole wheat pasta with chicken',
 'Quick tomato soup',
 'One-pot red pepper pasta with chicken']

In [79]:
recipes[recipes['rid'].isin(get_rec_id(recommendations))]

,publishdate,content_id,title_en,rid
330,2015-08-24,R121,Oven ratatouille with feta,330
610,2015-05-06,R203,Sweet potato curry,610
674,2017-04-05,R489,Four-cheese and tomato baked penne,674
783,2017-10-06,R822,One-pot spinach pasta,783
794,2018-03-29,R1164,One-pot red pepper pasta with chicken,794
883,2015-11-05,R341,Hot chickpea soup,883
901,2015-09-24,R302,Shakshuka,901
933,2014-11-26,R124,Creamy whole wheat pasta with chicken,933
1082,2017-01-11,R569,Quick tomato soup,1082
1698,2015-07-10,R260,Chili con carne,1698


In [80]:
item_liked = df[df.uid == 24687].rid.tolist()[:-10]
item_test = df[df.uid == 24687].rid.tolist()[-10:]
item_test

[1236, 1693, 783, 439, 595, 1775, 181, 883, 641, 878]

In [81]:
set(get_rec_id(recommendations)) & set(item_test)

{783, 883}

In [67]:
test.getrow(24687).indices

array([], dtype=int32)

In [ ]:
# Calculates a list of similar items
related = model_als.similar_items(4, N=10) # East African-style coconut chicken

In [ ]:
[related[i][0] for i in range(len(related))]

In [ ]:
similar_item = get_rec_title(related)
similar_item

In [ ]:
recipes[recipes['title_en'].isin(similar_item)]

In [ ]:
def precision_at_k_ran(test, user_items, k=10, user_index=None):
    if not user_index:
        user_index = range(user_items.shape[0])
    item_index = test.nonzero()[1]
    precisions = []
    for user in user_index:
        np.random.seed(1234)
        random_k = np.random.choice(item_index, replace=False,size=k).tolist()
        labels = test.getrow(user).indices
        precision = float(len(set(random_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)

In [32]:
##Here Recall@10 and Precision@10 is the same, since total # of relevant items = # of recommended items = 10
def precision_at_k(model,test, user_items, k=10, user_index=None):
    if not user_index:
        user_index = range(user_items.shape[0])
    user_items = user_items.tocsr()
    precisions = []
    for user in user_index:
        rec = model.recommend(user, user_items,  N=k, filter_already_liked_items= True)
        top_k = [rec[i][0] for i in range(len(rec))]
        labels = test.getrow(user).indices
        precision = float(len(set(top_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)


In [84]:
del precision_at_k

In [43]:
user_index[0:5]

[34899, 20983, 22232, 15600, 2931]

In [45]:
train.getrow(34899)
test.getrow(34899)

<1x1815 sparse matrix of type '<class 'numpy.float64'>'
	with 42 stored elements in Compressed Sparse Row format>

<1x1815 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [46]:
train_df.head()

,uid,idx
0,0,218
1,0,397
2,0,570
3,0,712
4,0,783


In [48]:
a = train_df.idx.unique()
b = test_df.idx.unique()

missing = []
for i in b:
  if i not in a:
    missing.append(i)

len(missing)

0

In [ ]:
precision_at_k_ran(test,train,10, user_index)

In [33]:
precision_at_k(model = model_als, test = test, user_items = train, k = 10, user_index = user_index) #regulazation = 0.01


0.05229566094853683

In [90]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors = 50, learning_rate = 0.01, regularization = 0.01, 
                                                         iterations = 100, verify_negative_samples = True)

In [91]:
model_bpr.fit(train.T) # training time 00:33

100%|██████████| 100/100 [00:33<00:00,  2.86it/s, correct=92.00%, skipped=5.34%]


In [ ]:
bpr_seq # why so slow??

In [ ]:
related = model_bpr.similar_items(4, N=10) # East African-style coconut chicken
[related[i][0] for i in range(len(related))]
similar_item = get_rec_title(related)
similar_item
recipes[recipes['title_en'].isin(similar_item)]

In [ ]:
type(recipes.publishdate[0])

In [88]:
len(test.nonzero()[0])
len(set(test.nonzero()[0]))

39640

3964

In [38]:
from implicit.evaluation import precision_at_k
p = precision_at_k(model_als, train, test,10)
p

100%|██████████| 39648/39648 [00:00<00:00, 47160.65it/s]


0.05229566094853683

In [86]:
from implicit.evaluation import precision_at_k
bpr = precision_at_k(model_bpr, train, test, 10)
bpr

100%|██████████| 39648/39648 [00:00<00:00, 45841.73it/s]


0.03796670030272452

In [40]:
from implicit.evaluation import mean_average_precision_at_k
map_10 = precision_at_k(model_als, train, test,10)
map_10

100%|██████████| 39648/39648 [00:00<00:00, 47053.56it/s]


0.05229566094853683

In [117]:
path = '/Users/xun/Documents/Thesis/'
train_new = pd.read_csv(path + 'interaction_data/ks-18m-cooks-v1_train_v3.csv')
test_new = pd.read_csv(path + 'interaction_data/ks-18m-cooks-v1_test_v3.csv')


In [118]:
I = train_new.uid.values
J = train_new.rid.values # turn mid to idx
V = np.ones(I.shape[0])
train_new_coo = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
train_new_csr = train_new_coo.tocsr()

I_t = test_new.uid.values
J_t = test_new.rid.values # turn mid to idx
V_t = np.ones(I_t.shape[0])
test_new_coo = sparse.coo_matrix((V_t, (I_t, J_t)), dtype=np.float64)
test_new_csr = test_new_coo.tocsr()

In [119]:
model_als = implicit.als.AlternatingLeastSquares(factors = 50, regularization = 0.01, 
                                                         iterations = 100)

# train is user_items sparse matrix, model.fit(item_users)
# training time cooks_18m : 02:09
model_als.fit(train_new_csr.T)

100%|██████████| 100.0/100 [02:00<00:00,  1.08s/it]


In [120]:
p = precision_at_k(model_als, train_new_csr, test_new_csr,10)
p

100%|██████████| 39563/39563 [00:00<00:00, 105995.71it/s]


0.05396458814472671

In [121]:
model_bpr.fit(train_new_csr.T)

100%|██████████| 100/100 [00:33<00:00,  3.03it/s, correct=95.09%, skipped=5.61%]


In [123]:
p_bpr = precision_at_k(model_bpr, train_new_csr, test_new_csr,10)
p_bpr

100%|██████████| 39563/39563 [00:00<00:00, 122774.91it/s]


0.06674364896073903

In [ ]:
del train_test_split

In [101]:
from implicit.evaluation import train_test_split as train_test_split_implicit

train, test = train_test_split_implicit(likes, 0.9)

In [102]:
train.shape
test.shape

(39648, 1815)

(39648, 1815)

In [109]:
len(np.unique(train.nonzero()[0]))
len(np.unique(train.nonzero()[1]))

39648

1743

In [110]:
len(np.unique(test.nonzero()[0]))
len(np.unique(test.nonzero()[1]))

32686

1741